In [170]:
import pandas as pd


csvPath =  "./Resources/purchase_data.csv"
itemPD_df = pd.read_csv(csvPath)

#Player count
playerCount = itemPD_df["SN"].nunique()

#Purchasing  Analysis (total)
#create series to store Purchasing Analysis data
pAnalysis = pd.Series({
    "Number of Unique Items":0,
    "Average Purchase Price": float(0),
    "Total Number of Purchases":0,
    "Total Revenue":0
})

#finds the number of unique items in the data set and stores it in series
pAnalysis["Number of Unique Items"] = itemPD_df["Item Name"].nunique()

#finds the mean of the price column from the data set and stores it in series
pAnalysis["Average Purchase Price"] = itemPD_df["Price"].mean()

#counts the total number of rows within the data set and stores that in series
pAnalysis["Total Number of Purchases"] = itemPD_df["Price"].count()

#adds price column together and stores it in series
pAnalysis["Total Revenue"] = itemPD_df["Price"].sum()

#Gender Demographics
#create series to store Gender Demographics
gDemo = pd.Series({
    "Male Players":0,
    "Male Player Percentage":0,
    "Female Players":0,
    "Female Player Percentage":0,
    "Other/Non-Disclosed Players":0,
    "Other/Non-Disclosed Player Percentage":0,
})

#remove duplicate players from df based on Screen Name
gDemoDupless_df = itemPD_df.drop_duplicates(subset="SN")

#sums the number of "Male" occurences in dupeless df then adds to series
gDemo["Male Players"] = (gDemoDupless_df.Gender == "Male").sum()

#Calculates percentage of players that are male
gDemo["Male Player Percentage"] = (gDemo["Male Players"] / playerCount) * 100

#sums the number of "Female" occurences in dupeless df then adds to series
gDemo["Female Players"] = (gDemoDupless_df.Gender == "Female").sum()

#Calculates percentage of players that are female
gDemo["Female Player Percentage"] = (gDemo["Female Players"] / playerCount) * 100

#sums the number of "Other/Non-Disclosed" occurences in dupeless df then adds to series
gDemo["Other/Non-Disclosed Players"] = (gDemoDupless_df.Gender == "Other / Non-Disclosed").sum()

#Calculates percentage of players that are Other/Non-Disclosed
gDemo["Other/Non-Disclosed Percentage"] = (gDemo["Other/Non-Disclosed Players"] / playerCount) * 100


#Purchasing Analysis(Gender)
#create DF to store purchasing analysis by gender
gPA_df = pd.DataFrame({"Gender":["Male", "Female","Other/Non-Disclosed"],
                      "Purchase Count":[0,0,0],
                      "Average Purchase Price":[0,0,0],
                      "Total Purchase Value":[0,0,0],
                      "Avg Total Purchase per Person":[0,0,0]    
                     })

#set index to be gender
gPA_df.set_index("Gender", inplace=True)

#
gPA_df.loc["Male","Purchase Count"] = itemPD_df.groupby('Gender')

print(gPA_df)

#top 5 spenders
topPurchaseGroup = itemPD_df.groupby("SN")[["Price"]].sum()
top5spenders = topPurchaseGroup.sort_values(["Price"], ascending=False)
top5spenders.head()



                     Purchase Count  Average Purchase Price  \
Gender                                                        
Male                              1                       0   
Female                            0                       0   
Other/Non-Disclosed               0                       0   

                     Total Purchase Value  Avg Total Purchase per Person  
Gender                                                                    
Male                                    0                              0  
Female                                  0                              0  
Other/Non-Disclosed                     0                              0  


,Price
SN,
Lisosia93,18.96
Idastidru52,15.45
Chamjask73,13.83
Iral74,13.62
Iskadarya95,13.10
